IMPORT das bibiliotecas de pyspark

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import SparkSession
import pyspark
import json
from pyspark.sql.functions import explode 
from pyspark.sql import functions


<H2> Chamando o dataframe no blob </H2>

In [ ]:
def chama_dataframe (path, tipo_arquivo = 'csv', header = True, inferSchema = True ):
##tipo arquivo = Json, CSV, Parquet etc
## se precisar pode por o .option('sep', ';')\
    try:
        df = spark.read\
            .format(tipo_arquivo)\
            .option('header', header)\
            .option('inferSchema', inferSchema)\
            .load(path)
    except:
        print("error")
        
    else:
        print("Sucesso, Vamos prosseguir")


Tratamentos de Colunas

In [ ]:
#Criando uma função Lambda parar tratar o header da coluna, removendo o espaço vazio e colocando um underscore. 
Nova_Lista= list(map(lambda x: x.replace(" ", "_"), df.columns))
df = d.toDF(*Nova_Lista)
#Abaixo todas as colunas com o nome maisculos
df = df.toDF(*[c.upper() for c in df.columns])

FUNÇÕES PRA TRATAMENTO

In [ ]:
#Trocar virgula por ponto OBS: você pode trocar por outras coisas tbm.
def convert_virgula(parametro):
    return df.withColumn(parametro, regexp_replace(parametro, ",", "."))


#Formatar a data 
def formatardata(parametro):
    return df.withColumn(parametro, to_date(parametro, 'dd/MM/yyyy'))

#Converter valor vazio em não informado OBS: Ou algo da sua escolha.
def convertevazio(df1, substitui_por='Não informado'):
    for i in df1.columns:
        df1 = df1.withColumn(i , when(col(i).isNull() | col(i).isin(''),substitui_por).otherwise(col(i)))
    return df1

#Converter valor nulo em não informado, é o mesmo do de cima...
def converternulo(df1, substitui_por='Não informado'):
    for i in df1.columns:
        df1 = df1.withColumn(i , when(col(i) == "", substitui_por).otherwise(col(i)))
    return df1

#Converter Valor null!
def converternulo2(df1, substitui_por='Não informado'):
    for i in df1.columns:
        df1 = df1.na.fill(substitui_por)
    return df1


#Funções de Count
#count nulos
def countnull(df):
    df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c)
               for c in df.columns]).display()

#count não nulos  
def countnotnull(df):
    df.select([count(when(isnan(c) | col(c).isNotNull(), c)).alias(c)
               for c in df.columns]).display()



<H3>Tipo de funções do proprio spark</H3>

In [ ]:
#withColumn()
"Tratar valores internos da coluna"
#Eu prefiro utilizar listas, feito o unico que esta situado abaixo.
df.withColumn('nomedacoluna', when(col('nomedacoluna').isin(unico), 'valorquemudarar').otherwise(col('nomedacoluna')))
    
#withColumnRenamed()
"Trata o nome das colunas"
df.withColumnRenamed("Antigo-nome", 'Novo-nome')

In [ ]:
class Conexao:
#classe conexão com funções para conectar ao banco SQL  

     #Informações para a conexão
    jdbcHostname = '<NOMEDOHOST>.database.windows.net'
    jdbcDatabase = '<NOME DO DB>'
    jdbcPort = 1433 #numero da porta
    jdbcUsername = '<USERNAME>'
    jdbcPassword = dbutils.secrets.get(scope = '<nome do escopo do KV>', key = '<nome da chave>')
    jdbcUrl = f'jdbc:sqlserver://{jdbcHostname}:{jdbcPort};database={jdbcDatabase}'
    
    def conectar(jdbcUsername=jdbcUsername, jdbcPassword = jdbcPassword ):
        #retorna informações de conexão com o SQL
        connectionProperties = {
          "user" : jdbcUsername,
          "password" : jdbcPassword,
          "driver" : "com.microsoft.sqlserver.jdbc.SQLServerDriver"
            }
        return connectionProperties
        
    def salvar(var, nome_tabela, schema, jdbcUrl=jdbcUrl,jdbcUsername = jdbcUsername, jdbcPassword = jdbcPassword):
        #salva um dataframe (var) com o nome schema.nome_tabela no banco de dados
        var.write.format('jdbc')\
                        .mode('overwrite')\
                        .option('url', jdbcUrl)\
                        .option('dbtable',  f'{schema}.{nome_tabela}' )\
                        .option("user", jdbcUsername) \
                        .option("password", jdbcPassword) \
                        .save()  
        
    def url_sql (jdbcUrl = jdbcUrl):
        #retorna somente a URL do banco, caso precise
        return jdbcUrl
    
    def retorna_df (nome_tabela_sql):
        #falta testar, deve funcionar
        conexao = Conexao.conectar()
        df = spark.read.jdbc(url=jdbcUrl, table=nome_tabela_sql, properties=conexao)
        return df

In [ ]:
### se quiser tratar dados pelo python
class Transform_lista:
  
    def coluna_to_lista (dataframe, str_coluna):
      #transforma um dataframe em uma lista
        list_cidade = dataframe.select([str_coluna]).rdd.flatMap(lambda x: x).collect()
        return list_cidade
    
    def lista_minuscula (lista): 
        #cria uma lista com letra minuscula
        lista_l = [str(i).lstrip() for i in lista]
        return lista_l
    
    def lista_unicos (lista, local=0, acento = False):
        #realiza a transformação tirando assentos e mantém somente um valor, em ordem alfabética
        cid = []
        for i in lista:
            if acento:
                a = unidecode(i)
            elif local != 0:
                a=i[local]
            else:
                a = i
            
            if a not in cid:
                cid.append(a)
        cid = sorted(cid)
        return cid
      
    def lista_eID(lista):
        #Cria uma lista com listas com id
        return [[i, lista[i]] for i in range(len(lista))]
    
    def del_null (lista):
        #deleta valores nulos de uma lista
        l_fix=[]
        for i in lista:
            linha = []
            for j in i:
                if j != None:
                    linha.append(j)
            l_fix.append(linha)
        return l_fix

Andando pelo Keyvault

In [ ]:
dbutils.secrets.listScopes() #lista todos os escopos do kv

dbutils.secrets.list('<escopo>') #lista as keys do escopo

key = dbutils.secrets.get(scope='<escopo>',key='<nome-chave>') #pega um valor de uma chave




mount e unmount no Azure/S3

In [ ]:
dbutils.fs.mount(
   source = f"wasbs://{nome_container}@{storage}.blob.core.windows.net",
   mount_point=f"/mnt/{nome_container}",
   extra_configs={f"fs.azure.account.key.{storage}.blob.core.windows.net":key})

dbutils.fs.unmount(f"/mnt/{nome_container}")


#S3 
aws_bucket_name = "<aws-bucket-name>"
mount_name = "<mount-name>"
dbutils.fs.mount(f"s3a://{aws_bucket_name}", f"/mnt/{mount_name}")
display(dbutils.fs.ls(f"/mnt/{mount_name}"))

Ingestão

In [1]:
def ingestion_table(table_name):
    query = import_query(f"sql/{table_name}.sql")
      
    (spark.sql(query)\
    .coalesce(1)\
    .write
    .mode('overwrite')
    .format("delta")
    .option("overwriteSchema", True)
    .saveAsTable(f"silver.{table_name}"))